# Isotherm processing

This notebook will process isotherms selected previously to generate KPI values for each one. To run this notebook the root directory must be the main folder.

In [1]:
import json
import pathlib
import pickle
from collections import Counter

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

import pygaps

In [114]:
# Database location
db_path = pathlib.Path.cwd() / "data" / "iso.db"

# Get all isotherms
isotherms = pygaps.db_get_isotherms(db_path, {})

print(f'Loaded {len(isotherms)} isotherms.')

Selected20913isotherms
Loaded 20913 isotherms.


Compute uptake values on pre-determined pressures. Do not extrapolate above maximum range. If any value below minimum range, use Henry constant to compute loading.

In [30]:
no_loading_possible = []
model_possible = []

for iso in tqdm(isotherms):

    iso.uptake = {}

    prange = np.arange(0.5, 20.5, 0.5)
    minp = min(iso.pressure(branch='ads'))
    maxp = max(iso.pressure(branch='ads'))
    model = [a for a in prange if a < minp]
    direct = [a for a in prange if minp < a < maxp]

    try:
        for p in direct:
            iso.uptake[p] = np.asscalar(iso.loading_at(p))
    except Exception:
        no_loading_possible.append(iso)
        continue
    # Take the data between 0 and x from the henry model
    if model:
        for p in model:
            iso.uptake[p] = iso.henry_k * p
        model_possible.append(iso)

    iso.uptake[0] = 0.  # we automatically include 0

0%|          | 0/20913 [00:00<?, ?it/s]C:\Users\pauli\Miniconda3\envs\dscience\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  app.launch_new_instance()
100%|██████████| 20913/20913 [01:53<00:00, 183.56it/s]


The calculated KPI are saved in a separate table in a HDF5 database. This will allow very fast retrieval when the dashboard is operated.

In [31]:
simple_dict = {}

for iso in tqdm(isotherms):

    addition = {
        'mat' : iso.material,
        'ads' : str(iso.adsorbate),
        't' : iso.temperature,
        'type' : iso.iso_type,
        'kH' : np.log(iso.henry_k),
    }
    for p in np.arange(0.5, 20.5, 0.5):

        addition[p] = iso.uptake.get(p, None)

    simple_dict[iso.filename] = addition

df = pd.DataFrame.from_dict(simple_dict, orient='index')
df.to_hdf(pathlib.Path.cwd() / 'data' / 'kpi.h5', 'table', mode='w')

100%|██████████| 20913/20913 [00:00<00:00, 40234.00it/s]
C:\Users\pauli\Miniconda3\envs\dscience\lib\site-packages\pandas\io\pytables.py:278: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
C:\Users\pauli\Miniconda3\envs\dscience\lib\site-packages\pandas\io\pytables.py:278: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items] [items->None]

  f(store)


For real isotherms to be displayed in the dashboard, some values are saved in an on-disk database. Here we use the python Shelve module for quick key-value pair storage.

In [129]:
import shelve

iso_packed = "./data/iso-packed"

with shelve.open(iso_packed) as packed_dict:
    for iso in tqdm(isotherms):
        packed_dict[iso.filename] = {
            'adsorbate': str(iso.adsorbate),
            'material': iso.material,
            'temp': iso.temperature,
            'x': iso.pressure(),
            'y': iso.loading(),
            'doi': iso.DOI,
        }


100%|██████████| 20913/20913 [03:25<00:00, 101.87it/s]


## New methods

In [2]:
df = pd.read_hdf(pathlib.Path.cwd() / 'data' / 'kpi.h5', 'table')

In [371]:
df.sample(n=500, random_state=1).to_hdf(pathlib.Path.cwd() / 'data' / 'kpi-smol.h5', 'table', mode='w')

C:\Users\pauli\Miniconda3\envs\dscience\lib\site-packages\pandas\io\pytables.py:278: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
C:\Users\pauli\Miniconda3\envs\dscience\lib\site-packages\pandas\io\pytables.py:278: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items] [items->None]

  f(store)


In [3]:
len(df['mat'].unique())

4413

In [3]:
# select on experiment
select = None
if select:
    dft = df[df['type']==select]
else:
    dft = df

# select on temperature
t_val = 303
t_mar = 10
dft = dft[dft['t'].between(t_val - t_mar, t_val + t_mar)]

# create two clean dataframes with only material
g1 = dft[dft['ads'] == 'nitrogen'].reset_index().drop(['index', 'type', 't', 'ads'], axis=1)
g2 = dft[dft['ads'] == 'methane'].reset_index().drop(['index', 'type', 't', 'ads'], axis=1)

# select only common materials
common = list(set(g1['mat'].unique()).intersection(g2['mat'].unique()))
g1 = g1[g1['mat'].isin(common)]
g2 = g2[g2['mat'].isin(common)]

print(len(g1))
print(len(g2))

554
694


In [95]:
from contextlib import contextmanager

def stats(series):

    no_nan = series.dropna()
    size = len(no_nan)

    if size == 0:
        med, std = np.nan, 0
    elif size == 1:
        med, std = float(no_nan), 0
    elif 1 < size <= 4:
        med, std = np.median(no_nan), np.std(no_nan)
    elif 4 < size:
        # Computing IQR
        Q3, Q1 = np.nanpercentile(sorted(no_nan), [75, 25], interpolation='linear')
        IQR = Q3 - Q1
        o_rem = no_nan[(Q1 - 1.5 * IQR < no_nan) | (no_nan > Q3 + 1.5 * IQR)]
        med, std = np.median(o_rem), np.std(o_rem)

    return size, med, std


def func(series):
    return pd.Series(stats(series), index=(["size", "med", "err"]), 
                     name=series.name)

@contextmanager
def _group_selection_context(groupby):
    """
    Set / reset the _group_selection_context.
    """
    groupby._set_group_selection()
    yield groupby
    groupby._reset_group_selection()

temp = None
def desc(data):
    return pd.concat([func(s) for _, s in data.items()], axis=1, sort=False)

def proc(data):
    with _group_selection_context(data):
        return data.apply(
            lambda x: pd.concat(
                [func(s) for _, s in x.items()], 
                axis=1, sort=False)
            ).unstack()

# %timeit proc(test.groupby('mat'))
# %timeit test.groupby('mat').agg(process)
# proc(test.groupby('mat'))

In [205]:
# res = proc(test.groupby('mat'))
final.loc['CuBTC', ('kH_x', 'size')]

59.0

In [78]:
test = pd.DataFrame({'mat': ['co2', 'co2', 'co2', 'n2', 'co2', 'n2', 'ch4', 'n2', 'co2', 'co2'],
                   'kH': np.random.randn(10),
                   'L': np.random.randn(10)})

In [5]:
dft[dft['ads'] == 'nitrogen'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)

In [99]:
import time

# start_time = time.time()

# final = pd.merge(
#     dft[dft['ads'] == 'nitrogen'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False).agg(stats),
#     dft[dft['ads'] == 'methane'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False).agg(stats),
#     on=('mat'), suffixes=('_x', '_y'))

# print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()

final = pd.merge(
    proc(dft[dft['ads'] == 'nitrogen'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)),
    proc(dft[dft['ads'] == 'methane'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)),
    on=('mat'), suffixes=('_x', '_y'))

print("--- %s seconds ---" % (time.time() - start_time))

--- 28.004876613616943 seconds ---


In [89]:
select = None
if select:
    sv = [select]
else:
    sv = ['unk', 'exp', 'sim']
t_abs = 303
t_tol = 10
g1 = 'nitrogen'
g2 = 'methane'

In [61]:
def method2():
    def single_g(gas):
        if select:
            return df[
                (df['type'] == select) &
                (df['t'].between(t_abs - t_tol, t_abs + t_tol)) &
                (df['ads'] == gas)
            ].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)
        else:
            return df[
                (df['t'].between(t_abs - t_tol, t_abs + t_tol)) &
                (df['ads'] == gas)
            ].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)

    return single_g(g1), single_g(g2)

In [94]:
def method3():
    if select:
        dft = df[
            (df['type'] == select) &
            (df['t'].between(t_abs - t_tol, t_abs + t_tol))
        ]
    else:
        dft = df[df['t'].between(t_abs - t_tol, t_abs + t_tol)]
        
    common = list(set(dft['mat'].unique()).intersection(dft['mat'].unique()))

    return (
        dft[(dft['ads'] == g1) & (dft['mat'].isin(common))].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False),
        dft[(dft['ads'] == g2) & (dft['mat'].isin(common))].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False),
    )

In [97]:
def method4():
    if select:
        dft = df[
            (df['type'] == select) &
            (df['t'].between(t_abs - t_tol, t_abs + t_tol))
        ]
    else:
        dft = df[df['t'].between(t_abs - t_tol, t_abs + t_tol)]
    
    g1_filt = dft[dft['ads'] == g1]
    g2_filt = dft[dft['ads'] == g2]
    common = list(set(g1_filt['mat'].unique()).intersection(g2_filt['mat'].unique()))

    return pd.merge(
        proc(g1_filt[g1_filt['mat'].isin(common)].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)),
        proc(g2_filt[g2_filt['mat'].isin(common)].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)),
        on=('mat'), suffixes=('_x', '_y'))
%timeit method4()

14.4 s ± 1.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
# grouped1 = dft[dft['ads'] == 'nitrogen'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)
# grouped2 = dft[dft['ads'] == 'methane'].drop(columns=['type', 't', 'ads']).groupby('mat', sort=False)

# common = list(set(grouped1.groups.keys()).intersection(grouped2.groups.keys()))


# print(len(grouped1.groups.keys()))
# print(len(grouped2.groups.keys()))

# print(len(common))
# len(grouped1.filter(lambda x: x.name in common).groups.keys())
%timeit method1()
%timeit method2()
%timeit method3()

10 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
10 ms ± 467 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.97 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [100]:
g1 = 'propane'
g2 = 'carbon dioxide'
ret = method4()

In [113]:
ret.loc['MIL-53(Cr)'].loc[('1.0_x', 'med')]


0.013044685266911256

## Old methods
Construct dictionary with results on a material basis.

In [7]:
materials = {}

gases = [
    'hydrogen', 'neon', 'argon', 'krypton', 'xenon',
    'methane', 'ethane', 'ethene', 'acetylene',
    'propane', 'propene',
    'butane', 'isobutane',
    '1-butene', 'cis-2-butene', 'trans-2-butene',
    'isobutene',
    'isopentane',
    'carbon dioxide', 'sulphur dioxide', 'nitrogen dioxide',
    'oxygen', 'carbon monoxide', 'nitrogen',
    'benzene', 'toluene',
    'water', 'methanol', 'ethanol', 'ammonia',
]

for mat in Counter([i.material for i in isotherms]).keys():
    materials[mat] = {
        gas: {
            'iso': [],
            'Kh': [],
            'L': []
        } for gas in gases}

for iso in isotherms:
    materials[iso.material][iso.adsorbate]['iso'].append(iso.filename)
    materials[iso.material][iso.adsorbate]['Kh'].append(iso.henry_slope)
    materials[iso.material][iso.adsorbate]['L'].append(iso.uptake)

Some outlier detection functions to be used later: gross outlier detection and interquartile outlier detection.

In [8]:
def kh_gross_outlier_rejection(a, l1=1e-7, l2=1e7):
    return [i for i in a if l1 < i < l2 and not np.isnan(i)]


def l_gross_outlier_rejection(a, l1=0, l2=1e7):
    return [i for i in a if l1 <= i < l2 and not np.isnan(i)]


def iqr_outlier_rejection(arr):
    q75, q25 = np.nanpercentile(sorted(arr), [75, 25], interpolation='linear')
    iqr = q75 - q25
    l_b = q25 - (1.5 * iqr)
    u_b = q75 + (1.5 * iqr)
    return [a for a in arr if a > l_b and a < u_b]

Outlier detection and median calculation for Henry constant.

In [9]:
for mat in materials:
    for gas in gases:
        K_hs = materials[mat][gas].get('Kh', None)
        if K_hs is None:
            materials[mat][gas]['mKh'] = np.nan
            materials[mat][gas]['eKh'] = np.nan
            materials[mat][gas]['lKh'] = np.nan

        K_hs = kh_gross_outlier_rejection(K_hs)

        if K_hs is None:
            materials[mat][gas]['mKh'] = np.nan
            materials[mat][gas]['eKh'] = np.nan
            materials[mat][gas]['lKh'] = np.nan

        nK_h = len(K_hs)
        if nK_h == 0:
            continue

        if nK_h > 4:
            K_hs = iqr_outlier_rejection(K_hs)
            mK_h = np.median(K_hs)
            eK_h = np.std(K_hs)
        elif nK_h == 1:
            mK_h = np.median(K_hs)
            eK_h = 0
        else:
            mK_h = np.median(K_hs)
            eK_h = np.std(K_hs)

        materials[mat][gas]['mKh'] = mK_h
        materials[mat][gas]['eKh'] = eK_h
        materials[mat][gas]['lKh'] = nK_h

Outlier detection and median calculation for uptake.

In [10]:
for mat in materials:
    for gas in gases:

        aL_s = materials[mat][gas].get('L', None)
        if not aL_s:
            materials[mat][gas]['mL'] = []
            materials[mat][gas]['eL'] = []
            materials[mat][gas]['lL'] = []
            continue

        pdL_s = pd.DataFrame(aL_s)

        materials[mat][gas]['mL'] = [0]
        materials[mat][gas]['eL'] = [0]
        materials[mat][gas]['lL'] = [0]

        for p in np.arange(0.5, 20.5, 0.5):
            try:
                L_s = l_gross_outlier_rejection(pdL_s[p])
                if L_s is None:
                    continue
            except Exception:
                continue

            nL_s = len(L_s)
            if nL_s == 0:
                materials[mat][gas]['mL'].append(np.nan)
                materials[mat][gas]['eL'].append(np.nan)
                materials[mat][gas]['lL'].append(np.nan)
                continue

            if nL_s > 4:
                L_s = iqr_outlier_rejection(L_s)
                mL_s = np.median(L_s)
                eL_s = np.std(L_s)
            elif nL_s == 1:
                mL_s = np.median(L_s)
                eL_s = 0
            else:
                mL_s = np.median(L_s)
                eL_s = np.std(L_s)

            materials[mat][gas]['mL'].append(mL_s)
            materials[mat][gas]['eL'].append(eL_s)
            materials[mat][gas]['lL'].append(nL_s)

Save the resulting json file.

In [11]:
save_path = r"./data/kpis.json"
with open(save_path, 'w') as file:
    json.dump(materials, file)